In [1]:
import numpy as np
import pandas as pd

In [2]:
dfRel = pd.read_csv('data/dfRel.csv',index_col=0)
dfUser = pd.read_table('data/dfUser.csv',sep=',')
truth = pd.read_csv('data/dfTruth.csv',index_col=0)

d:\Anaconda\envs\py37\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
# loc函数取targetUser==1的用户的id
targetUsers = dfUser.loc[dfUser['targetUser'] == 1, ['id']]

# 关联二度好友
df1 = pd.merge(targetUsers, dfRel, left_on=['id'], right_on=['userID'])
df2 = pd.merge(df1, dfRel, left_on=['friendID'], right_on=['userID'])
# 用group by计算共同好友数，形成分层索引；通过reset_index将series转为dataframe
df3 = df2.groupby(['id','friendID_y'])['friendID_x'].count().reset_index()
# 将结算结果改名为userID, ffID, cn
df3.rename(columns={'id':'userID','friendID_y':'ffID','friendID_x':'cn'}, inplace=True)
#print(df3.info())

# 补充用户信息
df4 = pd.merge(df3, dfUser, left_on='userID', right_on='id')
df5 = pd.merge(df4, dfUser, left_on='ffID', right_on='id')

# 关联好友表，用于标记二度好友是否为好友
df6 = pd.merge(df5, dfRel, how='left', left_on=['userID','ffID'], right_on=['userID','friendID'])
# 将标记改为0,1，并改名为isFriend，数据类型改为int
df6.loc[df6['friendID'].isnull(), ['friendID']]=0
df6.loc[df6['friendID']>0, ['friendID']]=1
df6.rename(columns={'friendID':'isFriend'}, inplace=True)
df6['isFriend']=df6['isFriend'].astype(int)

# 计算指标
df6['HPI'] = df6['cn']/df6.loc[:,['nFriends_x','nFriends_y']].T.max()
df6['Jaccard'] = df6['cn']/(df6.loc[:,['nFriends_x','nFriends_y']].T.sum()-df6['cn'])
df6['Sorenson'] = 2*df6['cn']/df6.loc[:,['nFriends_x','nFriends_y']].T.sum()
# print(df6.loc[df6['isFriend']>0].info())

In [4]:
df6

,userID,ffID,cn,Unnamed: 0_x,id_x,targetUser_x,fName_x,gender_x,starUser_x,mobileUser_x,...,nAlbums_y,nShortPosts_y,nVisits_y,location_y,privacy_y,timeline_y,isFriend,HPI,Jaccard,Sorenson
0,87702431,87656949,1,6232,87702431,1,王,1.0,1.0,0.0,...,1.0,78.0,6719.0,清华大学,1.0,0,0,0.000677,0.000553,0.001105
1,87808217,87656949,10,6595,87808217,1,张,1.0,1.0,1.0,...,1.0,78.0,6719.0,清华大学,1.0,0,0,0.006770,0.004537,0.009033
2,139840069,87656949,1,6940,139840069,1,叶,0.0,1.0,0.0,...,1.0,78.0,6719.0,清华大学,1.0,0,0,0.000677,0.000552,0.001103
3,150023465,87656949,1,7165,150023465,1,沈,1.0,1.0,0.0,...,1.0,78.0,6719.0,清华大学,1.0,0,0,0.000677,0.000549,0.001096
4,155712967,87656949,1,7296,155712967,1,吕,1.0,0.0,0.0,...,1.0,78.0,6719.0,清华大学,1.0,0,0,0.000677,0.000521,0.001042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205679,3195846773,3486811249,22,5780,3195846773,1,袁,1.0,1.0,0.0,...,0.0,1.0,324.0,吉林大学,1.0,0,1,0.037607,0.026829,0.052257
1205680,3195846773,3575901351,17,5780,3195846773,1,袁,1.0,1.0,0.0,...,0.0,3.0,277.0,吉林大学,1.0,0,1,0.029060,0.022049,0.043147
1205681,3195846773,3818278177,16,5780,3195846773,1,袁,1.0,1.0,0.0,...,3.0,7.0,756.0,吉林大学,1.0,0,1,0.027350,0.017699,0.034783
1205682,3195846773,4017986037,15,5780,3195846773,1,袁,1.0,1.0,0.0,...,0.0,1.0,256.0,吉林大学,1.0,0,1,0.025641,0.019557,0.038363


In [5]:
# 删除二度好友是自己
duplicate_list = []
for i in range(df6.shape[0]):
    if df6.iloc[i,0] == df6.iloc[i,1]:
        duplicate_list.append(i)

In [6]:
df6.drop(duplicate_list,axis=0,inplace=True)

In [7]:
# 剔除异常值
df6 = df6.loc[df6['Jaccard']>=0]
df6.dropna(axis=0,subset=['HPI'],inplace=True)

d:\Anaconda\envs\py37\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [8]:
# 归一化指标
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
temp = pd.DataFrame(scaler.fit_transform(df6.iloc[:,-3:]),columns=['scaled_HPI','scaled_Jaccard','scaled_Sorenson'])
df6.reset_index(inplace=True)
df6 = pd.concat([df6,temp],axis=1)

In [9]:
df6.set_index('index',inplace=True)

In [10]:
# df6 = df6.loc[df6['isFriend'] == 0]
df6['Similarity'] = (df6['scaled_HPI'] + df6['scaled_Jaccard'] + df6['scaled_Sorenson'])/3
df6 = df6.drop(['Unnamed: 0_x','id_x'],axis=1)

In [11]:
df6

,userID,ffID,cn,targetUser_x,fName_x,gender_x,starUser_x,mobileUser_x,homeProvince_x,homeCity_x,...,privacy_y,timeline_y,isFriend,HPI,Jaccard,Sorenson,scaled_HPI,scaled_Jaccard,scaled_Sorenson,Similarity
index,,,,,,,,,,,,,,,,,,,,,
0,87702431,87656949,1,1,王,1.0,1.0,0.0,江西,九江市,...,1.0,0,0,0.000677,0.000553,0.001105,0.001008,0.000274,0.000821,0.000701
1,87808217,87656949,10,1,张,1.0,1.0,1.0,陕西,西安市,...,1.0,0,0,0.006770,0.004537,0.009033,0.010148,0.002266,0.006768,0.006394
2,139840069,87656949,1,1,叶,0.0,1.0,0.0,安徽,黄山市,...,1.0,0,0,0.000677,0.000552,0.001103,0.001008,0.000273,0.000819,0.000700
3,150023465,87656949,1,1,沈,1.0,1.0,0.0,安徽,NaN,...,1.0,0,0,0.000677,0.000549,0.001096,0.001008,0.000272,0.000815,0.000698
4,155712967,87656949,1,1,吕,1.0,0.0,0.0,河南,商丘市,...,1.0,0,0,0.000677,0.000521,0.001042,0.001008,0.000258,0.000774,0.000680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205679,3195846773,3486811249,22,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,1,0.037607,0.026829,0.052257,0.056403,0.013412,0.039185,0.036333
1205680,3195846773,3575901351,17,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,1,0.029060,0.022049,0.043147,0.043583,0.011022,0.032353,0.028986
1205681,3195846773,3818278177,16,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,1,0.027350,0.017699,0.034783,0.041018,0.008847,0.026080,0.025315


In [13]:
df6.loc[df6["isFriend"]==1,:].to_csv("data/data2_2.csv")
df6.loc[df6["isFriend"]==0,:].to_csv("data/data_2.csv")